In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
from PIL import Image

In [0]:
from google.colab import files
import tensorflow as tf
import pandas as pd
import io
import numpy as np
import PIL
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

In [0]:
train = pd.read_csv('train.csv')
train_ids = train['Id'].values.tolist()
train_labels = train['DGCI'].values.tolist()
train_labels = np.array(train_labels,dtype=float).reshape([792,1])

test = pd.read_csv('sample.csv')
test_ids = test['Id'].values.tolist()

In [0]:
final = []
green = []
for label in train_ids:

    img = Image.open(str(label)+'.jpg')
    img = img.resize((40,30),PIL.Image.ANTIALIAS)
    x = np.array(img)
    #x = x.reshape(30,40,3)
    g = x[:,:,1].reshape(30,40,1)
    #g = g.reshape(300,400,1)
    final.append(x)
    green.append(g)

final = np.array(final,dtype=float)
green = np.array(green,dtype=float)
print(np.shape(green))

In [0]:
final_test = []
green_test = []
for label in test_ids:

    img = Image.open(str(label)+'.jpg')
    img = img.resize((40,30),PIL.Image.ANTIALIAS)
    x = np.array(img)
    #x = x.reshape(30,40,3)
    g = x[:,:,1].reshape(30,40,1)
    final_test.append(x)
    green_test.append(g)

final_test = np.array(final_test,dtype=float)
green_test = np.array(green_test,dtype=float)
print(np.shape(green_test))

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.callbacks import LearningRateScheduler
from keras.layers import Dense, Dropout, Activation, Flatten
import keras

In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (30, 40, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4)) 
model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1,activation='linear'))

In [0]:
model.compile(optimizer=keras.optimizers.Adadelta(), loss='mean_squared_error', metrics=['mean_squared_error'])

In [0]:
model.fit(final,train_labels,epochs=1,batch_size=64)

In [0]:
print(sqrt(model.evaluate(final,train_labels)[1]))

In [0]:
predicted = model.predict(final_test)

In [0]:
test['DGCI'] = predicted
test.to_csv('Predictions_DGCI.csv',index=False)

In [0]:
files.download('Predictions_DGCI.csv')